In [26]:
# Import necessary python libraries
import pandas as pd
import ast
import numpy as np
import re
import string

import random as rand
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [27]:
# tmdb_5000_movies.csv
mov_df1 = pd.read_csv('data/tmdb_5000_movies.csv')
mov_df1  # Test if csv is read

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

### Data Cleaning

In [28]:
# tmdb_5000_credits.csv
mov_df2 = pd.read_csv('data/tmdb_5000_credits.csv')
mov_df2.rename(columns = {'movie_id':'id'}, inplace = True) # Rename for merge purposes
mov_df2 # Test if csv is read

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [29]:
### Merge dataframes
dff = pd.merge(mov_df1, mov_df2, on = 'id', how = 'left')


# rewrite the original dataframes by the merged
df = dff[['id','title_x','genres','cast','production_companies','keywords']]
df.rename({'title_x':'title'}, axis=1)

,id,title,genres,cast,production_companies,keywords
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""id"": 5616, ""name"": ""united states\u2013mexi..."
4799,72766,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...","[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...",[],[]
4800,231617,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
4801,126186,Shanghai Calling,[],"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...",[],[]


In [30]:
# Clean data and dataframe

# We use 'title_x','cast_names','genres_list','prod_comps','keywords_names' 
def get_features(lst):
    feat = []
    for i in ast.literal_eval(lst):
        feat.append(i['name'])
    return feat


df['genres_list'] = df['genres'].apply(get_features)
df['prod_comps'] = df['production_companies'].apply(get_features)
df['keywords_names'] = df['keywords'].apply(get_features)
df['cast_names'] = df['cast'].apply(get_features)

# Create new clean dataframe to work with
clean_df = pd.DataFrame(df[['title_x','cast_names','genres_list','prod_comps','keywords_names']])

clean_df

<ipython-input-30-d4aa57b2ff68>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres_list'] = df['genres'].apply(get_features)
<ipython-input-30-d4aa57b2ff68>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prod_comps'] = df['production_companies'].apply(get_features)
<ipython-input-30-d4aa57b2ff68>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

,title_x,cast_names,genres_list,prod_comps,keywords_names
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Action, Adventure, Fantasy, Science Fiction]","[Ingenious Film Partners, Twentieth Century Fo...","[culture clash, future, space war, space colon..."
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Adventure, Fantasy, Action]","[Walt Disney Pictures, Jerry Bruckheimer Films...","[ocean, drug abuse, exotic island, east india ..."
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Action, Adventure, Crime]","[Columbia Pictures, Danjaq, B24]","[spy, based on novel, secret agent, sequel, mi..."
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Action, Crime, Drama, Thriller]","[Legendary Pictures, Warner Bros., DC Entertai...","[dc comics, crime fighter, terrorist, secret i..."
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Action, Adventure, Science Fiction]",[Walt Disney Pictures],"[based on novel, mars, medallion, space travel..."
...,...,...,...,...,...
4798,El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[Action, Crime, Thriller]",[Columbia Pictures],"[united states–mexico barrier, legs, arms, pap..."
4799,Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...","[Comedy, Romance]",[],[]
4800,"Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...","[Comedy, Drama, Romance, TV Movie]","[Front Street Pictures, Muse Entertainment Ent...","[date, love at first sight, narration, investi..."
4801,Shanghai Calling,"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",[],[],[]


In [33]:
# Rename dataframe indices
clean_df.rename(columns = {'title_x': 'title'}, inplace = True)
clean_df.index.rename('index', inplace = True)
clean_df

,title,cast_names,genres_list,prod_comps,keywords_names,raw_data
index,,,,,,
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Action, Adventure, Fantasy, Science Fiction]","[Ingenious Film Partners, Twentieth Century Fo...","[culture clash, future, space war, space colon...","[culture clash, future, space war, space colon..."
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Adventure, Fantasy, Action]","[Walt Disney Pictures, Jerry Bruckheimer Films...","[ocean, drug abuse, exotic island, east india ...","[ocean, drug abuse, exotic island, east india ..."
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Action, Adventure, Crime]","[Columbia Pictures, Danjaq, B24]","[spy, based on novel, secret agent, sequel, mi...","[spy, based on novel, secret agent, sequel, mi..."
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Action, Crime, Drama, Thriller]","[Legendary Pictures, Warner Bros., DC Entertai...","[dc comics, crime fighter, terrorist, secret i...","[dc comics, crime fighter, terrorist, secret i..."
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Action, Adventure, Science Fiction]",[Walt Disney Pictures],"[based on novel, mars, medallion, space travel...","[based on novel, mars, medallion, space travel..."
...,...,...,...,...,...,...
4798,El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[Action, Crime, Thriller]",[Columbia Pictures],"[united states–mexico barrier, legs, arms, pap...","[united states–mexico barrier, legs, arms, pap..."
4799,Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...","[Comedy, Romance]",[],[],"[Edward Burns, Kerry Bishé, Marsha Dietlein, C..."
4800,"Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...","[Comedy, Drama, Romance, TV Movie]","[Front Street Pictures, Muse Entertainment Ent...","[date, love at first sight, narration, investi...","[date, love at first sight, narration, investi..."


In [34]:
# Combine data for similarity measures
clean_df['raw_data'] = clean_df['keywords_names'] + clean_df['cast_names'] + clean_df['genres_list'] + clean_df['prod_comps']
clean_df

,title,cast_names,genres_list,prod_comps,keywords_names,raw_data
index,,,,,,
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Action, Adventure, Fantasy, Science Fiction]","[Ingenious Film Partners, Twentieth Century Fo...","[culture clash, future, space war, space colon...","[culture clash, future, space war, space colon..."
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Adventure, Fantasy, Action]","[Walt Disney Pictures, Jerry Bruckheimer Films...","[ocean, drug abuse, exotic island, east india ...","[ocean, drug abuse, exotic island, east india ..."
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Action, Adventure, Crime]","[Columbia Pictures, Danjaq, B24]","[spy, based on novel, secret agent, sequel, mi...","[spy, based on novel, secret agent, sequel, mi..."
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Action, Crime, Drama, Thriller]","[Legendary Pictures, Warner Bros., DC Entertai...","[dc comics, crime fighter, terrorist, secret i...","[dc comics, crime fighter, terrorist, secret i..."
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Action, Adventure, Science Fiction]",[Walt Disney Pictures],"[based on novel, mars, medallion, space travel...","[based on novel, mars, medallion, space travel..."
...,...,...,...,...,...,...
4798,El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[Action, Crime, Thriller]",[Columbia Pictures],"[united states–mexico barrier, legs, arms, pap...","[united states–mexico barrier, legs, arms, pap..."
4799,Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...","[Comedy, Romance]",[],[],"[Edward Burns, Kerry Bishé, Marsha Dietlein, C..."
4800,"Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...","[Comedy, Drama, Romance, TV Movie]","[Front Street Pictures, Muse Entertainment Ent...","[date, love at first sight, narration, investi...","[date, love at first sight, narration, investi..."


### Preprocessing 

In [35]:
# Combine data in clean_data column
clean_df['clean_data'] = [' '.join(map(str, l)) for l in clean_df['raw_data']]

# Tokenize data
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

clean_df['clean_data_tokenized'] = clean_df['clean_data'].apply(lambda x: tokenize(x.lower()))
clean_df

,title,cast_names,genres_list,prod_comps,keywords_names,raw_data,clean_data,clean_data_tokenized
index,,,,,,,,
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Action, Adventure, Fantasy, Science Fiction]","[Ingenious Film Partners, Twentieth Century Fo...","[culture clash, future, space war, space colon...","[culture clash, future, space war, space colon...",culture clash future space war space colony so...,"[culture, clash, future, space, war, space, co..."
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Adventure, Fantasy, Action]","[Walt Disney Pictures, Jerry Bruckheimer Films...","[ocean, drug abuse, exotic island, east india ...","[ocean, drug abuse, exotic island, east india ...",ocean drug abuse exotic island east india trad...,"[ocean, drug, abuse, exotic, island, east, ind..."
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Action, Adventure, Crime]","[Columbia Pictures, Danjaq, B24]","[spy, based on novel, secret agent, sequel, mi...","[spy, based on novel, secret agent, sequel, mi...",spy based on novel secret agent sequel mi6 bri...,"[spy, based, on, novel, secret, agent, sequel,..."
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Action, Crime, Drama, Thriller]","[Legendary Pictures, Warner Bros., DC Entertai...","[dc comics, crime fighter, terrorist, secret i...","[dc comics, crime fighter, terrorist, secret i...",dc comics crime fighter terrorist secret ident...,"[dc, comics, crime, fighter, terrorist, secret..."
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Action, Adventure, Science Fiction]",[Walt Disney Pictures],"[based on novel, mars, medallion, space travel...","[based on novel, mars, medallion, space travel...",based on novel mars medallion space travel pri...,"[based, on, novel, mars, medallion, space, tra..."
...,...,...,...,...,...,...,...,...
4798,El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[Action, Crime, Thriller]",[Columbia Pictures],"[united states–mexico barrier, legs, arms, pap...","[united states–mexico barrier, legs, arms, pap...",united states–mexico barrier legs arms paper k...,"[united, states, mexico, barrier, legs, arms, ..."
4799,Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...","[Comedy, Romance]",[],[],"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",Edward Burns Kerry Bishé Marsha Dietlein Caitl...,"[edward, burns, kerry, bishé, marsha, dietlein..."
4800,"Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...","[Comedy, Drama, Romance, TV Movie]","[Front Street Pictures, Muse Entertainment Ent...","[date, love at first sight, narration, investi...","[date, love at first sight, narration, investi...",date love at first sight narration investigati...,"[date, love, at, first, sight, narration, inve..."


In [36]:
# Define stop words list
stopwords = nltk.corpus.stopwords.words('english')     # All English Stopwords
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [37]:
# Removing stop words 

# Create function to remove stopwords
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopwords]
    return text

# Remove stop words from data
clean_df['clean_data_nostop'] = clean_df['clean_data_tokenized'].apply(lambda x: remove_stopwords(x))
clean_df

,title,cast_names,genres_list,prod_comps,keywords_names,raw_data,clean_data,clean_data_tokenized,clean_data_nostop
index,,,,,,,,,
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Action, Adventure, Fantasy, Science Fiction]","[Ingenious Film Partners, Twentieth Century Fo...","[culture clash, future, space war, space colon...","[culture clash, future, space war, space colon...",culture clash future space war space colony so...,"[culture, clash, future, space, war, space, co...","[culture, clash, future, space, war, space, co..."
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Adventure, Fantasy, Action]","[Walt Disney Pictures, Jerry Bruckheimer Films...","[ocean, drug abuse, exotic island, east india ...","[ocean, drug abuse, exotic island, east india ...",ocean drug abuse exotic island east india trad...,"[ocean, drug, abuse, exotic, island, east, ind...","[ocean, drug, abuse, exotic, island, east, ind..."
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Action, Adventure, Crime]","[Columbia Pictures, Danjaq, B24]","[spy, based on novel, secret agent, sequel, mi...","[spy, based on novel, secret agent, sequel, mi...",spy based on novel secret agent sequel mi6 bri...,"[spy, based, on, novel, secret, agent, sequel,...","[spy, based, novel, secret, agent, sequel, mi6..."
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Action, Crime, Drama, Thriller]","[Legendary Pictures, Warner Bros., DC Entertai...","[dc comics, crime fighter, terrorist, secret i...","[dc comics, crime fighter, terrorist, secret i...",dc comics crime fighter terrorist secret ident...,"[dc, comics, crime, fighter, terrorist, secret...","[dc, comics, crime, fighter, terrorist, secret..."
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Action, Adventure, Science Fiction]",[Walt Disney Pictures],"[based on novel, mars, medallion, space travel...","[based on novel, mars, medallion, space travel...",based on novel mars medallion space travel pri...,"[based, on, novel, mars, medallion, space, tra...","[based, novel, mars, medallion, space, travel,..."
...,...,...,...,...,...,...,...,...,...
4798,El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[Action, Crime, Thriller]",[Columbia Pictures],"[united states–mexico barrier, legs, arms, pap...","[united states–mexico barrier, legs, arms, pap...",united states–mexico barrier legs arms paper k...,"[united, states, mexico, barrier, legs, arms, ...","[united, states, mexico, barrier, legs, arms, ..."
4799,Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...","[Comedy, Romance]",[],[],"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",Edward Burns Kerry Bishé Marsha Dietlein Caitl...,"[edward, burns, kerry, bishé, marsha, dietlein...","[edward, burns, kerry, bishé, marsha, dietlein..."
4800,"Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...","[Comedy, Drama, Romance, TV Movie]","[Front Street Pictures, Muse Entertainment Ent...","[date, love at first sight, narration, investi...","[date, love at first sight, narration, investi...",date love at first sight narration investigati...,"[date, love, at, first, sight, narration, inve...","[date, love, first, sight, narration, investig..."


In [38]:
# Apply a lemmatizer
wn = nltk.WordNetLemmatizer()

# Create function to apply lematizer
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

# Apply lemmatizer
clean_df['clean_data_lemmatized'] = clean_df['clean_data_nostop'].apply(lambda x: lemmatizing(x))
clean_df.head()

,title,cast_names,genres_list,prod_comps,keywords_names,raw_data,clean_data,clean_data_tokenized,clean_data_nostop,clean_data_lemmatized
index,,,,,,,,,,
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Action, Adventure, Fantasy, Science Fiction]","[Ingenious Film Partners, Twentieth Century Fo...","[culture clash, future, space war, space colon...","[culture clash, future, space war, space colon...",culture clash future space war space colony so...,"[culture, clash, future, space, war, space, co...","[culture, clash, future, space, war, space, co...","[culture, clash, future, space, war, space, co..."
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Adventure, Fantasy, Action]","[Walt Disney Pictures, Jerry Bruckheimer Films...","[ocean, drug abuse, exotic island, east india ...","[ocean, drug abuse, exotic island, east india ...",ocean drug abuse exotic island east india trad...,"[ocean, drug, abuse, exotic, island, east, ind...","[ocean, drug, abuse, exotic, island, east, ind...","[ocean, drug, abuse, exotic, island, east, ind..."
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Action, Adventure, Crime]","[Columbia Pictures, Danjaq, B24]","[spy, based on novel, secret agent, sequel, mi...","[spy, based on novel, secret agent, sequel, mi...",spy based on novel secret agent sequel mi6 bri...,"[spy, based, on, novel, secret, agent, sequel,...","[spy, based, novel, secret, agent, sequel, mi6...","[spy, based, novel, secret, agent, sequel, mi6..."
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Action, Crime, Drama, Thriller]","[Legendary Pictures, Warner Bros., DC Entertai...","[dc comics, crime fighter, terrorist, secret i...","[dc comics, crime fighter, terrorist, secret i...",dc comics crime fighter terrorist secret ident...,"[dc, comics, crime, fighter, terrorist, secret...","[dc, comics, crime, fighter, terrorist, secret...","[dc, comic, crime, fighter, terrorist, secret,..."
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Action, Adventure, Science Fiction]",[Walt Disney Pictures],"[based on novel, mars, medallion, space travel...","[based on novel, mars, medallion, space travel...",based on novel mars medallion space travel pri...,"[based, on, novel, mars, medallion, space, tra...","[based, novel, mars, medallion, space, travel,...","[based, novel, mar, medallion, space, travel, ..."


In [39]:
# converting data
clean_df['clean_data_lemmatized'] = [' '.join(map(str, l)) for l in clean_df['clean_data_lemmatized']]
clean_df


,title,cast_names,genres_list,prod_comps,keywords_names,raw_data,clean_data,clean_data_tokenized,clean_data_nostop,clean_data_lemmatized
index,,,,,,,,,,
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Action, Adventure, Fantasy, Science Fiction]","[Ingenious Film Partners, Twentieth Century Fo...","[culture clash, future, space war, space colon...","[culture clash, future, space war, space colon...",culture clash future space war space colony so...,"[culture, clash, future, space, war, space, co...","[culture, clash, future, space, war, space, co...",culture clash future space war space colony so...
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Adventure, Fantasy, Action]","[Walt Disney Pictures, Jerry Bruckheimer Films...","[ocean, drug abuse, exotic island, east india ...","[ocean, drug abuse, exotic island, east india ...",ocean drug abuse exotic island east india trad...,"[ocean, drug, abuse, exotic, island, east, ind...","[ocean, drug, abuse, exotic, island, east, ind...",ocean drug abuse exotic island east india trad...
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Action, Adventure, Crime]","[Columbia Pictures, Danjaq, B24]","[spy, based on novel, secret agent, sequel, mi...","[spy, based on novel, secret agent, sequel, mi...",spy based on novel secret agent sequel mi6 bri...,"[spy, based, on, novel, secret, agent, sequel,...","[spy, based, novel, secret, agent, sequel, mi6...",spy based novel secret agent sequel mi6 britis...
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Action, Crime, Drama, Thriller]","[Legendary Pictures, Warner Bros., DC Entertai...","[dc comics, crime fighter, terrorist, secret i...","[dc comics, crime fighter, terrorist, secret i...",dc comics crime fighter terrorist secret ident...,"[dc, comics, crime, fighter, terrorist, secret...","[dc, comics, crime, fighter, terrorist, secret...",dc comic crime fighter terrorist secret identi...
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Action, Adventure, Science Fiction]",[Walt Disney Pictures],"[based on novel, mars, medallion, space travel...","[based on novel, mars, medallion, space travel...",based on novel mars medallion space travel pri...,"[based, on, novel, mars, medallion, space, tra...","[based, novel, mars, medallion, space, travel,...",based novel mar medallion space travel princes...
...,...,...,...,...,...,...,...,...,...,...
4798,El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[Action, Crime, Thriller]",[Columbia Pictures],"[united states–mexico barrier, legs, arms, pap...","[united states–mexico barrier, legs, arms, pap...",united states–mexico barrier legs arms paper k...,"[united, states, mexico, barrier, legs, arms, ...","[united, states, mexico, barrier, legs, arms, ...",united state mexico barrier leg arm paper knif...
4799,Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...","[Comedy, Romance]",[],[],"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",Edward Burns Kerry Bishé Marsha Dietlein Caitl...,"[edward, burns, kerry, bishé, marsha, dietlein...","[edward, burns, kerry, bishé, marsha, dietlein...",edward burn kerry bishé marsha dietlein caitli...
4800,"Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...","[Comedy, Drama, Romance, TV Movie]","[Front Street Pictures, Muse Entertainment Ent...","[date, love at first sight, narration, investi...","[date, love at first sight, narration, investi...",date love at first sight narration investigati...,"[date, love, at, first, sight, narration, inve...","[date, love, first, sight, narration, investig...",date love first sight narration investigation ...


### Vectorization

In [40]:
# Vectorize data using TF-IDF Vectorizer
tf = TfidfVectorizer()

# Create a matrix where we store the matrix containing each word and its TF-IDF score
tf_matrix = tf.fit_transform(clean_df['clean_data_lemmatized'])
tf_matrix

<4803x43193 sparse matrix of type '<class 'numpy.float64'>'
	with 301860 stored elements in Compressed Sparse Row format>

In [41]:
# Employ cosine similarity then view on a dataframe
cosine_sim = cosine_similarity(tf_matrix)
x = pd.DataFrame(cosine_sim)
x

,0,1,2,3,4,5,6,7,8,9,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.021780,0.012489,0.071068,0.088243,0.044370,0.022374,0.034595,0.027619,0.066422,...,0.007658,0.014710,0.036042,0.031004,0.000000,0.007295,0.003594,0.009436,0.025870,0.015876
1,0.021780,1.000000,0.017609,0.040973,0.035964,0.046224,0.033508,0.033623,0.016639,0.032054,...,0.004516,0.005667,0.020473,0.026080,0.000000,0.008672,0.000000,0.007725,0.009039,0.008454
2,0.012489,0.017609,1.000000,0.036819,0.010594,0.025789,0.004994,0.022504,0.022179,0.025440,...,0.001453,0.000000,0.005007,0.000000,0.000000,0.016936,0.000000,0.000934,0.005283,0.000000
3,0.071068,0.040973,0.036819,1.000000,0.042080,0.102071,0.017735,0.061331,0.048638,0.117846,...,0.008460,0.017330,0.035685,0.040062,0.007102,0.017588,0.009956,0.006716,0.021503,0.022161
4,0.088243,0.035964,0.010594,0.042080,1.000000,0.052737,0.050347,0.034058,0.012579,0.017467,...,0.000000,0.014483,0.011597,0.023481,0.000000,0.004898,0.000000,0.001651,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0.007295,0.008672,0.016936,0.017588,0.004898,0.005820,0.002079,0.001993,0.000000,0.008352,...,0.000000,0.007386,0.006608,0.026188,0.026177,1.000000,0.000000,0.002770,0.000000,0.000000
4799,0.003594,0.000000,0.000000,0.009956,0.000000,0.012572,0.000000,0.014557,0.000000,0.006028,...,0.000000,0.008867,0.000000,0.000000,0.000000,0.000000,1.000000,0.015145,0.000000,0.000000
4800,0.009436,0.007725,0.000934,0.006716,0.001651,0.025334,0.001961,0.010313,0.000000,0.006791,...,0.007371,0.039797,0.006202,0.009306,0.000000,0.002770,0.015145,1.000000,0.000000,0.019793
4801,0.025870,0.009039,0.005283,0.021503,0.000000,0.039871,0.000000,0.005915,0.019142,0.005686,...,0.000000,0.000000,0.018214,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.019497


In [42]:
# Function: Get index from title input
def get_index_from_title(title):
    idx_num = clean_df.index[clean_df['title'] == title]
    return idx_num

# Ask user for a title
title_test = input('Title: ')

# is the title in dataset
def movie_in_data(title):
    if title in df['title_x'].values:
        return True
    else:
        return False

# Function: Get recommendation of 5 similar movies
def get_movie_recommendation(title):
    # Store index in ind variable
    ind = get_index_from_title(title)

    # Run cosine similarity to find similar movies
    similar_mov_ind = cosine_sim[ind[0]].argsort()[:-7:-1]

    # Locate similar movies using indices
    # Then store similar movies in a list
    list_of_mov = []
    for x in similar_mov_ind:
        list_of_mov.append(clean_df.iloc[x]['title'])
    
    # Output
    return list_of_mov

print()
print("Top 5 Movie Recommendations")
print("---------------------------")
print("Because you liked: " + title_test)
for x in range(1,len(get_movie_recommendation(title_test))):
    print(str(x) + ". " + get_movie_recommendation(title_test)[x])

Title: Frozen

Top 5 Movie Recommendations
---------------------------
Because you liked: Frozen
1. Wreck-It Ralph
2. Big Hero 6
3. Return to Never Land
4. Enchanted
5. Winnie the Pooh
